<a href="https://colab.research.google.com/github/Gadugcc/Hiperparametros_SVM/blob/main/Hiperpar%C3%A2metros_Regress%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Lendo a base de dados

df = pd.read_csv("/content/drive/MyDrive/Recursos Estudos Colab/Otimização parametros/housing.csv",sep=',',encoding='iso-8859-1')

df.head()

,RM,LSTAT,PTRATIO,MEDV
0,6.575,4.98,15.3,504000.0
1,6.421,9.14,17.8,453600.0
2,7.185,4.03,17.8,728700.0
3,6.998,2.94,18.7,701400.0
4,7.147,5.33,18.7,760200.0


**Atributos Previsores**

RM: é o número médio de cômodos entre os imóveis no bairro.

LSTAT: é aporcentagem de proprietarios no bairro considerados de "classe baixa"

PTRATIO: é a razão entre estudantes e professores nas escolas de ensino fundamental e médio no bairro

**Variável Alvo**

MEDV: valor médio das casas

In [4]:
# tamanho do arquivo
df.shape

(489, 4)

# Exploração e tratamento dos dados

In [5]:
# Verificando se tem valor nulo
df.isnull().sum()

,0
RM,0
LSTAT,0
PTRATIO,0
MEDV,0


In [7]:
# Análise dos atributos
df.dtypes

,0
RM,float64
LSTAT,float64
PTRATIO,float64
MEDV,float64


# Análise dos tipos de atributos


In [8]:
import plotly.express as px

In [10]:
box = px.box(df,y='RM')
box.show()

In [11]:
box = px.box(df,y='LSTAT')
box.show()

In [12]:
box = px.box(df,y='PTRATIO')
box.show()

In [13]:
box = px.box(df,y='MEDV')
box.show()